## Install

In [1]:
!pip3 install huggingface-hub

In [2]:
!pip install --upgrade openai
!pip install --upgrade pydantic

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/07/de/ef3534d9417f7c72c75036fae6c85d9071aebbce8aa3616d3e69b9f0ca4d/openai-1.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/39/9b/4937d841aee9c2c8102d9a4eeb800c7dad25386caabb4a1bf5010df81a57/httpx-0.26.0-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions<5,>=4.7 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore==1.* from https://files.pythonhosted.org/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9840b6202a93d9a0f/httpcore-1.0.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.2 MB/s eta 0:00:00
   ━

In [3]:
!pip install numpy

In [4]:
!pip install fake-useragent

  Obtaining dependency information for fake-useragent from https://files.pythonhosted.org/packages/33/c9/ff44922639b8827dbc86d463d870dabfc19d1567d8a6427dcb2289d83fd8/fake_useragent-1.4.0-py3-none-any.whl.metadata


In [5]:
!pip install --upgrade openai
!pip install --upgrade pydantic

## Import

In [6]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_gemini = user_secrets.get_secret("GEMINI_API_KEY")
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
# secret_wandb = user_secrets.get_secret("wandb")

In [7]:
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
import os
import json
import re
import time
import random

In [9]:
import numpy as np

## Common function

In [10]:
timeGlobal = 0
def startTime():
    global timeGlobal
    timeGlobal = time.time()
def getTime():
    return (time.time() - timeGlobal)

## Download dataset

In [11]:
!mkdir dataset

In [12]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="hotamago/ZAIC-2023", filename="Elementary Maths Solving/test.zip", revision="main", repo_type="dataset", local_dir="dataset", local_dir_use_symlinks=False)
hf_hub_download(repo_id="hotamago/ZAIC-2023", filename="Elementary Maths Solving/train.zip", revision="main", repo_type="dataset", local_dir="dataset", local_dir_use_symlinks=False)

'dataset/Elementary Maths Solving/train.zip'

In [13]:
!sudo apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [14]:
!mkdir datasetRaw
!unzip -q -o "dataset/Elementary Maths Solving/test.zip" -d "datasetRaw"
!unzip -q -o "dataset/Elementary Maths Solving/train.zip" -d "datasetRaw"

In [15]:
hf_hub_download(repo_id="hotamago/ZAIC-2023", filename="Elementary Maths Solving/math_train_short.json", revision="main", repo_type="dataset", local_dir="datasetRaw", local_dir_use_symlinks=False)

'datasetRaw/Elementary Maths Solving/math_train_short.json'

In [16]:
!mv "/kaggle/working/datasetRaw/Elementary Maths Solving/math_train_short.json" "/kaggle/working/datasetRaw/"

## Load dataset

In [17]:
train_data = None
test_data = None
with open(os.path.join("datasetRaw", "train", "/kaggle/working/datasetRaw/math_train.json"), "r") as f:
# with open(os.path.join("datasetRaw", "train", "/kaggle/working/datasetRaw/math_train_short.json"), "r") as f:
    train_data = json.loads(f.read())["data"]
with open(os.path.join("datasetRaw", "test", "/kaggle/working/datasetRaw/math_test.json"), "r") as f:
    test_data = json.loads(f.read())['data']

In [18]:
train_data[0]

{'id': '1',
 'question': 'Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính',
 'choices': ['A. 80,000 : 6',
  'B. 80,000 x 6',
  'C. 80,000 : (6 x 100)',
  'D. (80,000 x 6) : 100'],
 'explanation': 'Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồng . Để tìm số tiền lỗ ta có thể lấy 80 000 chia cho 100 rồi nhân với 6 (tức là 80 000 : 100 × 6) hoặc lấy 80000 nhân với 6 rồi chia cho 100 (tức là 80 000 × 6 : 100).',
 'answer': 'D. (80,000 x 6) : 100'}

In [19]:
test_data[0]

{'id': '01-0203',
 'question': 'Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?',
 'choices': ['A. 4 500 000 đồng',
  'B. 45 000 000 đồng',
  'C. 50 000 000 đồng',
  'D. 450 000 000 đồng']}

## Preprocess

In [20]:
MAX_TOKEN_MODEL = 512

## Auto gen probelm

### Init

In [ ]:
class AutoDataset:
    def __init__(self, *args, **kwargs):
        self.dataset = []
    
    def add(self, data):
        self.dataset.append({
            'id': 'autodataset-{0}'.format(len(self.dataset)),
            **data
        })
        
    def get(self):
        return self.dataset
    
    def size(self):
        return len(self.dataset)
    
    def clear(self):
        self.dataset = []

In [ ]:
autodataset = AutoDataset()

### Function math

In [ ]:
DICT_FUN = {
    "sort": lambda x: [float(y) for y in x].sort(),
}

### Class

In [ ]:
class BaseGenChapter:
    def __init__(self, *args, **kwargs):
        pass
    
    def _call(self, *args, **kwargs) -> str:
        raise NotImplementedError("_call not Implement yet")
    
    def call(self, *args, **kwargs) -> str:
        data = self._call(*args, **kwargs)
        return {
            "question": data[0],
            "choices": data[1],
            "answer": data[2],
            "formula": data[3],
        }

In [ ]:
# Biggest Smallest number problem
class BSNumberProblem(BaseGenChapter):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def _call(self):
        listNumber = np.random.uniform(low=-1000, high=1000, size=(random.randint(2, 6)))
        listNumber = [round(i, 3) for i in listNumber]
        mode = random.choice(["B", "S"])
        if mode == "B":
            trueNumber = str(np.amax(listNumber))
            mode_str = "lớn"
        elif mode == "S":
            trueNumber = str(np.amin(listNumber))
            mode_str = "nhỏ"
            
        listNumber = [str(i) for i in listNumber]
        
        strListNumber = ", ".join(listNumber)
        
        question = f"Số {mode_str} nhất trong các số sau {strListNumber} là:"
        choices = [f"{chr(i + ord('A'))}. " for i in range(len(listNumber))]
        
        posTrueAnswerRan = random.randint(0, len(choices) - 1)
        choices[posTrueAnswerRan] += trueNumber
        answer = choices[posTrueAnswerRan]
        
        listNumberNoTrue = [i for i in listNumber if i != trueNumber]
        random.shuffle(listNumberNoTrue)
        temp_choices = []
        temp_index = 0
        for i in range(len(choices)):
            if i == posTrueAnswerRan:
                temp_choices.append(choices[i])
            else:
                temp_choices.append(f"{choices[i]}{listNumberNoTrue[temp_index]}")
                temp_index+=1
        choices = temp_choices
        
        listNumberFloat = [float(i) for i in listNumber]
        if mode == "B":
            formula = "max(" + str(listNumberFloat) + ")"
        elif mode == "S":
            formula = "min(" + str(listNumberFloat) + ")"
        
        return [question, choices, answer, formula]

In [ ]:
def autoCreateAnswerChoices(listEle, mark, formula, delta=5):
    trueNumber = eval(formula.format(*listEle))
    listNumber = np.random.uniform(low=-delta, high=delta, size=(random.randint(2, 6)))
    listNumber += trueNumber
    listNumber = [round(i, 3) for i in listNumber]
    
    trueNumber = str(trueNumber)
    listNumber = [str(i) for i in listNumber]
    
    choices = [f"{chr(i + ord('A'))}. " for i in range(len(listNumber))]
    posTrueAnswerRan = random.randint(0, len(choices) - 1)
    choices[posTrueAnswerRan] += trueNumber
    answer = choices[posTrueAnswerRan]
    
    listNumberNoTrue = [i for i in listNumber if i != trueNumber]
    random.shuffle(listNumberNoTrue)
    temp_choices = []
    temp_index = 0
    for i in range(len(choices)):
        if i == posTrueAnswerRan:
            temp_choices.append(choices[i])
        else:
            temp_choices.append(f"{choices[i]}{listNumberNoTrue[temp_index]}")
            temp_index+=1
    choices = temp_choices
    
    return [answer, choices]

In [ ]:
# Convert problem
class ConvertProblem(BaseGenChapter):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def _call(self):
        common_theme = "{inp_theme} . Số điền vào chỗ trống là:"
        dict_type = {
            "m2dm": {
                "theme": "{0}m = ...dm",
                "formula": "{0} * 10",
                "num": 2,
                "mask": [1]
            },
            "m2cm": {
                "theme": "{0}m = ...cm",
                "formula": "{0} * 100",
                "num": 2,
                "mask": [1]
            },
            "cm2m": {
                "theme": "{0}cm = ...m",
                "formula": "{0} / 100",
                "num": 2,
                "mask": [1]
            },
            "cm2dm": {
                "theme": "{0}cm = ...dm",
                "formula": "{0} / 10",
                "num": 2,
                "mask": [1]
            },
            "mdm2cm": {
                "theme": "{0}m {1}dm = ...cm",
                "formula": "{0} * 100 + {1} * 10",
                "num": 3,
                "mask": [2]
            },
            "m2mcm-get-cm": {
                "theme": "{0}m = {1}m ...cm",
                "formula": "({0} - {1}) * 100",
                "num": 3,
                "mask": [2]
            },
            "m2dmcm-get-dm": {
                "theme": "{0}m = ...dm {2}cm",
                "formula": "{0} * 10 - {2} / 10",
                "num": 3,
                "mask": [1]
            },
            "mdmcm2mm": {
                "theme": "{0}m {1}dm {2}cm = ...mm",
                "formula": "{0} * 1000 + {1} * 100 + {2} * 10",
                "num": 4,
                "mask": [3]
            },
            "phut2gio": {
                "theme": "{0} phút = ...giờ",
                "formula": "{0} / 60",
                "num": 2,
                "mask": [1]
            },
            "gio2phut": {
                "theme": "{0} giờ = ...phút",
                "formula": "{0} * 60",
                "num": 2,
                "mask": [1]
            },
            "giophut2phut": {
                "theme": "{0} giờ {1} phút = ...phút",
                "formula": "{0} * 60 + {1}",
                "num": 3,
                "mask": [2]
            },
            "phut2giophut": {
                "theme": "{0} phút = ...giờ {2} phút",
                "formula": "({0} - {2}) / 60",
                "num": 3,
                "mask": [1]
            },
            "m2km": {
                "theme": "{0}m = ...km",
                "formula": "{0} / 1000",
                "num": 2,
                "mask": [1]
            },
            "m22km2": {
                "theme": "{0}m2 = ...km2",
                "formula": "{0} / 1000000",
                "num": 2,
                "mask": [1]
            },
            "m22ha": {
                "theme": "{0}m2 = ...ha",
                "formula": "{0} / 10000",
                "num": 2,
                "mask": [1]
            },
            "m22dm2": {
                "theme": "{0}m2 = ...dm2",
                "formula": "{0} * 100",
                "num": 2,
                "mask": [1]
            },
            "m32dm3": {
                "theme": "{0}m3 = ...dm3",
                "formula": "{0} * 1000",
                "num": 2,
                "mask": [1]
            }
        }
        mode_type = random.choice(list(dict_type.values()))
        listVal = np.random.randint(1, 100, (mode_type["num"]))
        question_temp = mode_type["theme"].format(*listVal)
        question = common_theme.format(inp_theme = question_temp)
        answer, choices = autoCreateAnswerChoices(listVal, mode_type["mask"], mode_type["formula"])
        return [question, choices, answer, mode_type["formula"].format(*listVal)]

In [ ]:
# Caculate problem
class CaculateProblem(BaseGenChapter):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def _call(self):
        common_theme = "{inp_theme} . Số điền vào chỗ trống là:"
        dict_type = {
            "0": {
                "theme": "({0} + {1}) * {2} = ...",
                "formula": "({0} + {1}) * {2}",
                "num": 4,
                "mask": [3]
            },
            "1": {
                "theme": "({0} + {1}) / {2} = ...",
                "formula": "({0} + {1}) / {2}",
                "num": 4,
                "mask": [3]
            },
            "2": {
                "theme": "{0} + {1} * {2} = ...",
                "formula": "{0} + {1} * {2}",
                "num": 4,
                "mask": [3]
            },
            "3": {
                "theme": "{0} - {1} = ...",
                "formula": "{0} - {1}",
                "num": 4,
                "mask": [3]
            },
            "4": {
                "theme": "{0} + {1} + {2} = ...",
                "formula": "{0} + {1} + {2}",
                "num": 4,
                "mask": [3]
            },
            "5": {
                "theme": "{0} - {1} + {2} = ...",
                "formula": "{0} - {1} + {2}",
                "num": 4,
                "mask": [3]
            },
        }
        mode_type = random.choice(list(dict_type.values()))
        listVal = np.random.uniform(-10000, 10000, (mode_type["num"]))
        listVal = [round(i, 3) for i in listVal]
        question_temp = mode_type["theme"].format(*listVal)
        question = common_theme.format(inp_theme = question_temp)
        answer, choices = autoCreateAnswerChoices(listVal, mode_type["mask"], mode_type["formula"], 10000)
        return [question, choices, answer, mode_type["formula"].format(*listVal)]

In [ ]:
class MangerAutoGen:
    def __init__(self, listGener = []):
        self.listGener = listGener
        for gener in listGener:
            if not isinstance(gener, BaseGenChapter):
                raise TypeError("gener need to be instance of BaseGenChapter")
    
    def call(self):
        posRan = random.randint(0, len(self.listGener) - 1)
        return self.listGener[posRan].call()

In [ ]:
mangerAutoGen = MangerAutoGen([CaculateProblem(), ConvertProblem(), BSNumberProblem()])

In [ ]:
for i in range(1000):
    autodataset.add(mangerAutoGen.call())

#### Checking dataset

In [ ]:
datasetTemp = autodataset.get()
for i in range(len(datasetTemp)):
    singleData = datasetTemp[i]
    try:
        outFormula = eval(singleData["formula"])
    except Exception as e:
        print(f"Error formula testcase ({i})")
        print(singleData)
        print(type(e))
        print(e)
        
        break
    
    answer = float(singleData["answer"][3:])
    if answer != outFormula:
        print(f"Error answer testcase ({i})")
        print(singleData)
        print(f"out formula: {outFormula} vs answer: {answer}")
        break

In [ ]:
with open(os.path.join("result.json"), "w", encoding='utf-8') as f:
    json.dump(autodataset.get(), f)

### Add Formula to dataset

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key=user_secrets.get_secret("OPENAI_API_KEY"),
#     organization='org-j48waUrvSOM1n0J1SLXiAr8n',
)

def autoGPTFormula(instruction, explanation, answer):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": (
                "Below is an instruction that describes a task. Write a formula in eval function python that caculate the answer. No explanation. No words\n"
                "Formula must runable in python eval function\n"
                "Formula must don't have variable, number only\n"
                "Formula eval in python must have result equal with answer below\n"
                "Example formula: 3 - 3 * sqrt(10)"
            )
        },
        {"role": "user", "content": f"### Instruction:\n{instruction}\n\n### Explanation:\n{explanation}\n\n### Answer:\n{answer}"},
        {"role": "system", "content": "### Formula: "},
      ],
#       max_tokens=512,
      temperature=0,
      top_p=1.0,
#       top_k=50,
    )
    return response.choices[0].message.content

In [ ]:
import json
import requests
import hashlib
import string
import random
from fake_useragent import UserAgent

class DeepAIChatCompletion:
    @classmethod
    def md5(self, text):
        return hashlib.md5(text.encode()).hexdigest()[::-1]

    @classmethod
    def get_api_key(self, user_agent):
        part1 = str(random.randint(0, 10**11))
        part2 = self.md5(user_agent+self.md5(user_agent+self.md5(user_agent+part1+"x")))
        return f"tryit-{part1}-{part2}"

    @classmethod
    def create(self, messages):
        user_agent = UserAgent().random
        # api_key = self.get_api_key(user_agent)
        api_key = user_secrets.get_secret("DEEPAI_KEY")
        headers = {
          "api-key": api_key,
          "user-agent": user_agent
        }
        files = {
          "chat_style": (None, "chat"),
          "chatHistory": (None, json.dumps(messages))
        }

        r = requests.post("https://api.deepai.org/hacking_is_a_serious_crime", headers=headers, files=files, stream=True)

        for chunk in r.iter_content(chunk_size=None):
            r.raise_for_status()
            yield chunk.decode()

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key=user_secrets.get_secret("OPENAI_API_KEY"),
#     organization='org-j48waUrvSOM1n0J1SLXiAr8n',
)

def autoDeepAIFormula(instruction, explanation, answer):
    messages=[
        {"role": "system", "content": (
                "Below is an instruction that describes a task. Write a formula in eval function python that caculate the answer. No explanation. No words\n"
                "Formula must runable in python eval function\n"
                "Formula must don't have variable, number only\n"
                "Formula eval in python must have result equal with answer below\n"
                "Example formula: 3 - 3 * sqrt(10)"
            )
        },
        {"role": "user", "content": f"### Instruction:\n{instruction}\n\n### Explanation:\n{explanation}\n\n### Answer:\n{answer}"},
        {"role": "system", "content": "### Formula: "},
      ]
    
    temp_str = ""
    for chunk in DeepAIChatCompletion.create(messages):
        temp_str += chunk
    return temp_str

In [25]:
from math import *

## Add explantion and vetify

In [248]:
pip install google-generativeai

  Obtaining dependency information for google-generativeai from https://files.pythonhosted.org/packages/b5/7f/35f89209487f8473edc9d2cecef894a54680cf666e32893a767d12a8dba9/google_generativeai-0.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for google-ai-generativelanguage==0.4.0 from https://files.pythonhosted.org/packages/40/c2/d28988d3cba74e712f47a498e2b3e3b58ac215106019bf5d8c20f8ab9822/google_ai_generativelanguage-0.4.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.9/146.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 27.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [249]:
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GEMINI_API_KEY")
genai.configure(api_key = api_key)

# Generate content
modelGemini = genai.GenerativeModel(
    model_name='gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=0.0,
        top_p=0.4,
        top_k=50,
#         max_output_tokens=512,
    )
)
# response = modelGemini.generate_content('Tell me a story about a magic backpack.')
# print(response.text)

In [28]:
from math import *
import math

In [33]:
def autoGPTAddExplantion(problem, choices, answer):
    message = """Write a explanation for answer of the problem.

## Problem
{problem}

## Choices
{choices}

## True answer
{answer}

## Rule of explanation
- No markdown format
- Short explanation and step-by-step
- Few words, instead of using words, use math formulas instead
- Explan by Vietnamese step-by-step

# Response must follow format bellow

## Explanation
{{explanation}}""".format(problem = problem, choices = "\n".join(choices), answer = answer)
    
    return modelGemini.generate_content(message).text

i = 0
startTime()
cntSt = 0
timeWait = 1
while i < len(train_data):
    singleData = train_data[i]
    if "explanation" in singleData.keys() and type(singleData["explanation"]) is str:
        i += 1
        continue
    print(f"Runing testcase ({i})")
    
    try:
        res = autoGPTAddExplantion(singleData['question'], singleData['choices'], singleData['answer'])
        resRex = re.findall("## Explanation\n([\S\s]+)|## Giải thích\n([\S\s]+)", res)
        if len(resRex) == 0:
            outputExplanation = res
        else:
            outputExplanation = "".join(resRex[0])
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        print(res)
        
        time.sleep(timeWait)
        timeWait *= 2
        if timeWait > 8:
            break
        print
        continue
    
    timeWait = 1
        
    cntSt += 1
    
    train_data[i]['explanation'] = outputExplanation
    print(f"Done testcase ({i})")
#     print(resRex)
    print(train_data[i])
#     print(outputExplanation)
#     break;
    
    if cntSt%60 == 0:
        deltaTime = getTime()
        if (60 - deltaTime) > -1:
            time.sleep(60 - deltaTime + 2)
        startTime()
        cntSt = 0
    
    i += 1
print("Done all testcase")

Runing testcase (1109)
Done testcase (1109)
{'id': '891', 'question': 'Một cửa hàng trong một tháng đã bán hết 17400 bút bi. Hỏi trong tháng đó cửa hàng đã bán được bao nhiêu tá bút bi? (mỗi tá gồm 12 cái)', 'choices': ['A. 1350 tá', 'B. 1400 tá', 'C. 1450 tá', 'D. 1440 tá'], 'answer': 'C. 1450 tá', 'explanation': 'Số tá bút bi cửa hàng bán được trong tháng đó là:\n\n17400 : 12 = 1450 (tá)\n\nVậy cửa hàng đã bán được 1450 tá bút bi trong tháng đó.'}
Runing testcase (1110)
Done testcase (1110)
{'id': '892', 'question': 'Một cửa hàng tuần thứ nhất bán được 62941 ki lô gam gạo. Tuần thứ hai họ bán được ít hơn tuần thứ nhất 20142 ki lô gam gạo. Số ki lô gam gạo cửa hàng đã bán được trong tuần thứ hai là:', 'choices': ['A. 42395kg', 'B. 42957kg', 'C. 42799kg', 'D. 48294kg'], 'answer': 'C. 42799kg', 'explanation': 'Số ki lô gam gạo cửa hàng đã bán được trong tuần thứ hai là:\n\n62941 - 20142 = 42799 (kg)\n\nVậy đáp án đúng là C. 42799kg'}
Runing testcase (1111)
Done testcase (1111)
{'id': '8

In [45]:
train_data_verified = []

In [39]:
import json

In [47]:
# Verify sample
def autoGPTVerifySample(problem, choices, explanation, answer):
    message = """Verify sample which include [problem description, choise, explantion, true answer] are 
solvable problem, correct answer, correct explanation. Write a response follow the rule

## Problem
{problem}

## Choices
{choices}

## Explanation
{explanation}

## True answer
{answer}

## Rule of response
- No markdown format
- Response must be json
- Response type value must follow the format
- Response must follow format bellow

## Response format
{{
"solvable": {{boolean}}
"answer": {{boolean}}
"explanation": {{boolean}}
}}""".format(problem = problem, choices = "\n".join(choices), explanation = explanation, answer = answer)
    
    return modelGemini.generate_content(message).text

i = len(train_data_verified)
startTime()
cntSt = 0
timeWait = 1
while i < len(train_data):
    singleData = train_data[i]
    
    print(f"Runing testcase ({i})")
    
    outVerify = False
    
    try:
        res = autoGPTVerifySample(singleData['question'], singleData['choices'], singleData['explanation'], singleData['answer'])
        resRex = re.findall("```json\n([\S\s]+)\n```", res)
        if len(resRex) > 0:
            res = resRex[0]
        
        jsonRes = json.loads(res)
        if ("solvable" not in jsonRes) and ("answer" not in jsonRes) and ("explanation" not in jsonRes):
            raise Exception("Miss keys in res")
        if jsonRes["solvable"] and jsonRes["answer"] and jsonRes["explanation"]:
            outVerify = True
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        print(res)
        
        time.sleep(timeWait)
        timeWait *= 2
        if timeWait > 32:
            break
        continue
    
    timeWait = 1
        
    cntSt += 1
    
    train_data_verified.append({
        "ok": outVerify,
        **singleData
    })
    
    if not outVerify:
        print("--- False ---")
        print(res)
        print("--- False ---")
    
    print(singleData)
    
    print(f"Done testcase ({i})")
    
    if cntSt%60 == 0:
        deltaTime = getTime()
        if (60 - deltaTime) > -1:
            time.sleep(60 - deltaTime + 2)
        startTime()
        cntSt = 0
    
    i += 1
print("Done all testcase")

Runing testcase (268)
{'id': '1680', 'question': 'Trong các phép tính sau, phép tính có kết quả lớn nhất là:', 'choices': ['A. 1 215 × 5', 'B. 8 084 : 2', 'C. 7 527 : 3', 'D. 800 × 4'], 'explanation': 'So sánh kết quả của các phép tính ta được: 2 509 < 3 200 < 4 042 < 6 075.\n Vậy phép tính có kết quả lớn nhất là: 1 215 × 5', 'answer': 'A. 1 215 × 5'}
Done testcase (268)
Runing testcase (269)
{'id': '1682', 'question': 'Trong các phép tính sau, phép tính có kết quả nhỏ nhất là:', 'choices': ['A. 90 155 : 5', 'B. 13 456 + 9 528', 'C. 5218 ${\\times}$ 3', 'D. 20 590 – 1 565'], 'explanation': '90 155 : 5 = 18 031\n 13 456 + 9 528 = 22 984\n 5 2l8 ${\\times}$ 3 = 15 654 \n 20 590 – 1 565 = 19 025\n Phép tính có kết quả nhỏ nhất là: 5 218 ${\\times}$ 3', 'answer': 'C. 5218 ${\\times}$ 3'}
Done testcase (269)
Runing testcase (270)
{'id': '1683', 'question': 'Trong các phép tính sau, phép tính có kết quả nhỏ nhất là:', 'choices': ['A.9 000 + 7 500', 'B. 20 000 – 3 000', 'C. 4 500 ${\\ti

In [48]:
with open("/kaggle/working/datasetRaw/math_train_verified.json", "w", encoding='utf-8') as f:
    json.dump({
        "__count__": len(train_data_verified),
        "data": train_data_verified
    }, f)

## Add more sample

In [58]:
def getRandomSample():
    sample = {"ok" : False}
    while not sample["ok"]:
        sample = random.choice(train_data_verified)
    return sample

def checkTrueAnswerInChoices(sample):
    answer = sample["answer"]
    choices = sample["choices"]
    if answer in choices:
        return True
    return False

In [60]:
getRandomSample()

{'ok': True,
 'id': '462',
 'question': 'Đoạn đường trước cửa nhà Huy dài 50 m. Bố Huy trồng một số cây xà cừ, mỗi cây cách nhau 5 m. Số cây bố Huy trồng được là:',
 'choices': ['A. 11 cây', 'B. 10 cây', 'C. 9 cây', 'D. 8 cây'],
 'explanation': 'Số cây bố Huy trồng được là: 50 : $5 + 1 = 11$ (cây)',
 'answer': 'A. 11 cây'}

In [71]:
train_data_gen = []

In [80]:
modelGemini = genai.GenerativeModel(
    model_name='gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=0.0,
        top_p=0.7,
        top_k=40,
#         max_output_tokens=512,
    )
)

In [82]:
# Verify sample
def autoGPTGenSample(problem, choices, explanation, answer):
    message = """Generate a sample follow rule and based on the given sample

## Problem
{problem}

## Choices
{choices}

## Explanation
{explanation}

## True answer
{answer}

## Rule of response
- No markdown format
- Response must be json
- Response type value must follow the format
- Response must follow format bellow
- Sample must include [question, explanation, choices (There must be only 1 correct choice, another must wrong), true answer]
- Sample must be a math problem
- Sample only changes numbers in given sample
- Explanation in sample must few words, instead of using words, use math formulas instead
- Explanation in sample must short and step-by-step

## Response format
{{
"question": {{string}},
"explanation": {{string}},
"choices": [
{{string}},
{{string}},
{{string}},
{{string}}
],
"answer": {{string}}
}}""".format(problem = problem, choices = "\n".join(choices), explanation = explanation, answer = answer)
    
    return modelGemini.generate_content(message).text

i = len(train_data_gen)
startTime()
cntSt = 0
timeWait = 1
while i < 5000:
    singleData = getRandomSample()
    
#     res = autoGPTGenSample(singleData['question'], singleData['choices'], singleData['explanation'], singleData['answer'])
#     print(singleData)
#     print(res)
#     print(checkTrueAnswerInChoices(json.loads(res)))
#     break
    
    print(f"Runing testcase ({i})")
    
    try:
        res = autoGPTGenSample(singleData['question'], singleData['choices'], singleData['explanation'], singleData['answer'])
        resRex = re.findall("```json\n([\S\s]+)\n```", res)
        if len(resRex) > 0:
            res = resRex[0]
        
        jsonRes = json.loads(res)
        if ("question" not in jsonRes) and ("explanation" not in jsonRes) and ("answer" not in jsonRes) and ("choices" not in jsonRes):
            raise Exception("Miss keys in res")
        if len(jsonRes["choices"]) != 4:
            raise Exception("Choices keys len != 4")
            
        if not checkTrueAnswerInChoices(jsonRes):
            ansRex = re.findall("[ABCD]\. ?([\S\s]+)", jsonRes['answer'])
            if len(ansRex) > 0:
                jsonRes['answer'] = ansRex[0]
            if not checkTrueAnswerInChoices(jsonRes):
                raise Exception("No true answer in choices")
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        print(res)
        
        time.sleep(timeWait)
        timeWait *= 2
        if timeWait > 32:
            break
        continue
    
    timeWait = 1
        
    cntSt += 1
    
    train_data_gen.append({
        **jsonRes
    })
    
    print("--- Gen ---")
    print(jsonRes)
    print("--- End gen ---")
    print(singleData)
    
    print(f"Done testcase ({i})")
    
    if cntSt%60 == 0:
        deltaTime = getTime()
        if (60 - deltaTime) > -1:
            time.sleep(60 - deltaTime + 2)
        startTime()
        cntSt = 0
    
    i += 1
print("Done all testcase")

--- Gen ---
{'question': 'Sôý thích hợp điển vào chổ chẩm 7kg 50g = …….. kg là:', 'explanation': '50g = 0,05 kg\n7 kg 50g = 7 kg + 0,05 kg = 7,05 kg\nVậy số thích hợp cần điển vào chổ chẩm là: 7,05', 'choices': ['75', '70,5', '7,05', '7,5'], 'answer': '7,05'}
--- End gen ---
{'ok': True, 'id': '1486', 'question': 'Số thích hợp điền vào chỗ chấm 5kg 30g = …….. kg là:', 'choices': ['A. 53', 'B. 50,3', 'C. 5,03', 'D. 5,3'], 'explanation': '30g = 0,03 kg5 kg 30g = 5 kg + 0,03 kg = 5,03 kg Vậy số thích hợp cần điền vào chỗ chấm là: 5,03', 'answer': 'C. 5,03'}
Done testcase (2781)
Runing testcase (2782)
--- Gen ---
{'question': '12 là tích của hai số nào?', 'explanation': '2 ${\times}$ 6 = 12', 'choices': ['A. 2 và 6', 'B. 4 và 3', 'C. 5 và 2', 'D. 5 và 1'], 'answer': 'A. 2 và 6'}
--- End gen ---
{'ok': True, 'id': '189', 'question': '8 là tích của hai số nào?', 'choices': ['A. 2 và 5', 'B. 4 và 2', 'C. 5 và 3', 'D. 5 và 1'], 'explanation': 'Ta có: 2 ${\\times}$ 5 = 10 \n 4 ${\

KeyboardInterrupt: 

In [83]:
train_data_gen_verified = []

In [85]:
# Verify sample
def autoGPTVerifySample(problem, choices, explanation, answer):
    message = """Verify sample which include [problem description, choise, explantion, true answer] are 
solvable problem, correct answer, correct explanation. Write a response follow the rule

## Problem
{problem}

## Choices
{choices}

## Explanation
{explanation}

## True answer
{answer}

## Rule of response
- No markdown format
- Response must be json
- Response type value must follow the format
- Response must follow format bellow

## Response format
{{
"solvable": {{boolean}}
"answer": {{boolean}}
"explanation": {{boolean}}
}}""".format(problem = problem, choices = "\n".join(choices), explanation = explanation, answer = answer)
    
    return modelGemini.generate_content(message).text

i = len(train_data_gen_verified)
startTime()
cntSt = 0
timeWait = 1
while i < len(train_data_gen):
    singleData = train_data_gen[i]
    
    print(f"Runing testcase ({i})")
    
    outVerify = False
    
    try:
        res = autoGPTVerifySample(singleData['question'], singleData['choices'], singleData['explanation'], singleData['answer'])
        resRex = re.findall("```json\n([\S\s]+)\n```", res)
        if len(resRex) > 0:
            res = resRex[0]
        
        jsonRes = json.loads(res)
        if ("solvable" not in jsonRes) and ("answer" not in jsonRes) and ("explanation" not in jsonRes):
            raise Exception("Miss keys in res")
        if jsonRes["solvable"] and jsonRes["answer"] and jsonRes["explanation"]:
            outVerify = True
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        print(res)
        
        time.sleep(timeWait)
        timeWait *= 2
        if timeWait > 32:
            break
        continue
    
    timeWait = 1
        
    cntSt += 1
    
    train_data_gen_verified.append({
        "ok": outVerify,
        **singleData
    })
    
    if not outVerify:
        print("--- False ---")
        print(res)
        print("--- False ---")
    
    print(singleData)
    
    print(f"Done testcase ({i})")
    
    if cntSt%60 == 0:
        deltaTime = getTime()
        if (60 - deltaTime) > -1:
            time.sleep(60 - deltaTime + 2)
        startTime()
        cntSt = 0
    
    i += 1
print("Done all testcase")

{'question': 'Một hình thoi có độ dài đường chéo thứ nhất là 15 cm và đường chéo thứ hai có độ dài bằng $\x0crac{3}{5}$ đường chéo thứ nhất. Diện tích hình thoi là:', 'explanation': 'Độ dài đường chéo thứ hai là: 15 ${\times}$ $\x0crac{3}{5}$ = 9 (cm) \n Diện tích hình thoi là: 15 ${\times}$ 9 : 2 = 67,5 (cm2)', 'choices': ['A. 33,75 cm2', 'B. 45 cm2', 'C.67,5 cm2', 'D. 90 cm2'], 'answer': 'C.67,5 cm2'}
Done testcase (2659)
Runing testcase (2660)
{'question': 'Hình bình hành có đặc điểm gì?', 'explanation': '- Hình bình hành có 4 cạnh, 2 cặp cạnh đối diện song song và bằng nhau.\n- Hình bình hành có 2 cặp góc đối diện bằng nhau.\n- Hình bình hành có 2 đường chéo cắt nhau tại trung điểm của mỗi đường chéo.', 'choices': ['A. Hai cặp cạnh đối diện vuông góc', 'B. Hai cặp cạnh đối diện cắt nhau', 'C. Hai cặp cạnh đối diện song song và bằng nhau', 'D. Không có đặc điểm nhận biết'], 'answer': 'C. Hai cặp cạnh đối diện song song và bằng nhau'}
Done testcase (2660)
Runing test

In [86]:
with open("/kaggle/working/datasetRaw/math_train_gen_verified.json", "w", encoding='utf-8') as f:
    json.dump({
        "__count__": len(train_data_gen_verified),
        "data": train_data_gen_verified
    }, f)

In [92]:
while 1:
    time.sleep(1)
    print("True")

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 

## Add formula

In [21]:
hf_hub_download(repo_id="hotamago/ZAIC-2023", filename="Elementary Maths Solving/math_train_gen_verified.json", revision="main", repo_type="dataset", local_dir="datasetRaw", local_dir_use_symlinks=False)
os.system('mv "/kaggle/working/datasetRaw/Elementary Maths Solving/math_train_gen_verified.json" "/kaggle/working/datasetRaw/"')
with open("/kaggle/working/datasetRaw/math_train_gen_verified.json", "r") as f:
    train_data_gen_verified = json.loads(f.read())['data']

In [29]:
hf_hub_download(repo_id="hotamago/ZAIC-2023", filename="Elementary Maths Solving/math_train_verified.json", revision="main", repo_type="dataset", local_dir="datasetRaw", local_dir_use_symlinks=False)
os.system('mv "/kaggle/working/datasetRaw/Elementary Maths Solving/math_train_verified.json" "/kaggle/working/datasetRaw/"')
with open("/kaggle/working/datasetRaw/math_train_verified.json", "r") as f:
    train_data_verified = json.loads(f.read())['data']

In [30]:
train_data_mix_verified = [
    *train_data_verified,
    *train_data_gen_verified
]

In [31]:
len(train_data_mix_verified)

4338

In [39]:
train_data_mix_verified_no = [i for i in train_data_mix_verified if i["ok"] == True]

In [42]:
modelGemini = genai.GenerativeModel(
    model_name='gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=0.0,
        top_p=0.4,
        top_k=50,
#         max_output_tokens=128,
    )
)

In [43]:
train_data_full = []

In [34]:
import math
import numpy
import datetime
import itertools

In [44]:
def autoGPTAddFormula(problem, choices, explanation, oldformula, evaloldformula):
    errorMess = """- Don't make mistake like old expression
    
## Old expression
{oldformula}

## Error of old expression
{evaloldformula}
""".format(oldformula = oldformula, evaloldformula = evaloldformula)
    
    if len(oldformula) == 0 or len(evaloldformula) == 0:
        errorMess = ""
    
    message = """Write a python expression follow the rule that analytics problem or help to caculate the answer of the problem.

## Problem
{problem}

## Choices
{choices}

## Explanation
{explanation}

## Rule of expression:
- Must be python expression not python script
- No need to import library
- Only use function in math library (gcd, sqrt, max, min, comb, etc..) and numpy library
- Expression must in 1 line
- Must no standard input
- Example: 3 + math.sqrt(2)
{errorMess}
# Response format
## Expression
{{expression}}""".format(problem = problem, choices = "\n".join(choices), explanation = explanation, errorMess = errorMess)
    
    return modelGemini.generate_content(message).text

# formula

i = len(train_data_full)
startTime()
cntSt = 0
timeWait = 1
oldformula = []
evaloldformula = []
while i < len(train_data_mix_verified):
    singleData = train_data_mix_verified[i]
    
    print(f"Runing testcase ({i})")
    
    try:
        res = autoGPTAddFormula(singleData['question'], singleData['choices'], singleData['explanation'], oldformula, evaloldformula)
        outputFormula = res
        
        resRex = re.findall("## Expression\n([\S\s]+)", res)
        if len(resRex) > 0:
            outputFormula = resRex[0]
        
        outputFormulaRex = re.findall("```python\n([\S\s]*)\n```", outputFormula)
        if len(outputFormulaRex) > 0:
            outputFormula = outputFormulaRex[0]
        
        eval(outputFormula)
        
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        print(res)
        
        oldformula.append(outputFormula)
        evaloldformula.append("Expression not invalid: {0}".format(inst))
        
        time.sleep(timeWait)
        timeWait *= 2
        if timeWait > 8:
            train_data_full.append({
                **singleData,
                "formula": "error"
            })
            train_data_full[len(train_data_full) - 1]["ok"] = False
            timeWait = 1
            oldformula = []
            evaloldformula = []
            print(f"Done error testcase ({i})")
            i += 1
#             break
        continue
    
    timeWait = 1
        
    cntSt += 1
    
    oldformula = []
    evaloldformula = []
    
    train_data_full.append({
        **singleData,
        "formula": outputFormula
    })
    
    print("--- Gen ---")
    print(outputFormula, " = ", eval(outputFormula))
    print("--- End gen ---")
    
#     print(singleData)
    
    print(f"Done testcase ({i})")
    
    if cntSt%60 == 0:
        deltaTime = getTime()
        if (60 - deltaTime) > -1:
            time.sleep(60 - deltaTime + 2)
        startTime()
        cntSt = 0
    
    i += 1
print("Done all testcase")

--- Gen ---
max(33, 21, 17, 13), min(33, 21, 17, 13), sorted([33, 21, 17, 13])[1], sorted([33, 21, 17, 13])[2]  =  (33, 13, 17, 21)
--- End gen ---
Done testcase (2087)
Runing testcase (2088)
--- Gen ---
numpy.remainder(554, 5)  =  4
--- End gen ---
Done testcase (2088)
Runing testcase (2089)
--- Gen ---
math.pi * (3/2)**2  =  7.0685834705770345
--- End gen ---
Done testcase (2089)
Runing testcase (2090)
--- Gen ---
15 - 7  =  8
--- End gen ---
Done testcase (2090)
Runing testcase (2091)
--- Gen ---
min(124, 178, 187, 241)  =  124
--- End gen ---
Done testcase (2091)
Runing testcase (2092)
--- Gen ---
math.ceil(120.25 / 42.5)  =  3
--- End gen ---
Done testcase (2092)
Runing testcase (2093)
--- Gen ---
45 * 1000 + 600 + 70 + 8  =  45678
--- End gen ---
Done testcase (2093)
Runing testcase (2094)
--- Gen ---
69345 + 7218 / 3  =  71751.0
--- End gen ---
Done testcase (2094)
Runing testcase (2095)
--- Gen ---
56  =  56
--- End gen ---
Done testcase (2095)
Runing testcase (2096)
--- Gen --

In [45]:
with open("/kaggle/working/datasetRaw/math_train_full.json", "w", encoding='utf-8') as f:
    json.dump({
        "__count__": len(train_data_full),
        "data": train_data_full
    }, f)

In [49]:
train_data_full[4232]

{'ok': True,
 'question': 'Một con thỏ có thể chạy với vận tốc 6,5m/giây. Một con ngựa có thể chạy với vận tốc 20,2km/giờ. Hỏi trong 1 phút, con nào di chuyển được quãng đường dài hơn và dài hơn bao nhiêu mét?',
 'explanation': 'Đổi 20,2km/giờ = 5,6m/giây\nTrong 1 phút, con thỏ chạy được: 6,5 x 60 = 390 (m)\nTrong 1 phút, con ngựa chạy được: 5,6 x 60 = 336 (m)\nVậy con thỏ di chuyển được quãng đường dài hơn và dài hơn con ngựa là: 390 - 336 = 54 (m)',
 'choices': ['Con thỏ; 54m',
  'Con ngựa; 54m',
  'Con ngựa; 540m',
  'Con thỏ; 540m'],
 'answer': 'Con thỏ; 54m',
 'formula': '(6.5 * 60) - (5.6 * 60)'}

In [48]:
len([i for i in train_data_full if i["ok"] == True])

4207

In [29]:
import sys
from io import StringIO
import contextlib

@contextlib.contextmanager
def stdoutIO(stdout=None):
    old = sys.stdout
    if stdout is None:
        stdout = StringIO()
    sys.stdout = stdout
    yield stdout
    sys.stdout = old

## Verify

In [ ]:
hf_hub_download(repo_id="hotamago/ZAIC-2023", filename="Elementary Maths Solving/math_train_full.json", revision="main", repo_type="dataset", local_dir="datasetRaw", local_dir_use_symlinks=False)
os.system('mv "/kaggle/working/datasetRaw/Elementary Maths Solving/math_train_full.json" "/kaggle/working/datasetRaw/"')
with open("/kaggle/working/datasetRaw/math_train_full.json", "r") as f:
    train_data_full = json.loads(f.read())['data']

In [64]:
train_data_full_verified = []

In [65]:
# Verify sample
def autoGPTVerifySampleSuper(problem, choices, explanation, expression, expressionResult, answer):
    message = """Verify sample which include [problem description, choise, explantion, true answer, expression, eval expression result] are solvable problem (math problem can solve with logic), correct answer, correct explanation, correct expression (helpful expression, eval expression result provides useful information or aids in calculating the answer). Write a response follow the rule

## Problem
{problem}

## Choices
{choices}

## Explanation
{explanation}

## Expression
{expression}

## Eval expression result
{expressionResult}

## True answer
{answer}

## Rule of response
- No markdown format
- Response must be json
- Response type value must follow the format
- Response must follow format bellow

## Response format
{{
"problem": {{boolean}},
"answer": {{boolean}},
"explanation": {{boolean}},
"expression": {{boolean}}
}}""".format(
        problem = problem,
        choices = "\n".join(choices),
        explanation = explanation,
        expression = expression,
        expressionResult = expressionResult,
        answer = answer
    )
    
    return modelGemini.generate_content(message).text

listKeysNeed = [
    "problem", "answer", "explanation", "expression"
]

i = len(train_data_full_verified)
startTime()
cntSt = 0
timeWait = 1
while i < len(train_data_full):
    singleData = train_data_full[i]
    
    if not singleData["ok"]:
        train_data_full_verified.append(singleData)
        i += 1
        continue
    
    print(f"Runing testcase ({i})")
    
    outVerify = False
    
    try:
        res = autoGPTVerifySampleSuper(singleData['question'], singleData['choices'], singleData['explanation'], singleData['formula'], eval(singleData['formula']), singleData['answer'])
        resRex = re.findall("```json\n([\S\s]+)\n```", res)
        if len(resRex) > 0:
            res = resRex[0]
        
        jsonRes = json.loads(res)
        
        outVerify = True
        for j in range(len(listKeysNeed)):
            if (listKeysNeed[j] not in jsonRes):
                raise Exception("Miss keys in res")
            if not jsonRes[listKeysNeed[j]]:
                outVerify = False
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        print(res)
        
        time.sleep(timeWait)
        timeWait *= 2
        if timeWait > 32:
            break
        continue
    
    timeWait = 1
        
    cntSt += 1
    
    train_data_full_verified.append(singleData)
    train_data_full_verified[-1]["ok"] = outVerify
    
    if not outVerify:
        print("--- False ---")
        print(res)
        print("--- False ---")
        print(json.dumps(singleData, indent=4))
        train_data_full_verified[-1]["okChecker"] = jsonRes
    
    print(f"Done testcase ({i})")
    
    if cntSt%60 == 0:
        deltaTime = getTime()
        if (60 - deltaTime) > -1:
            time.sleep(60 - deltaTime + 2)
        startTime()
        cntSt = 0
    
    i += 1
print("Done all testcase")

Done testcase (2244)
Runing testcase (2245)
Done testcase (2245)
Runing testcase (2246)
Done testcase (2246)
Runing testcase (2247)
Done testcase (2247)
Runing testcase (2249)
Done testcase (2249)
Runing testcase (2250)
Done testcase (2250)
Runing testcase (2251)
Done testcase (2251)
Runing testcase (2252)
Done testcase (2252)
Runing testcase (2253)
Done testcase (2253)
Runing testcase (2254)
Done testcase (2254)
Runing testcase (2256)
Done testcase (2256)
Runing testcase (2257)
Done testcase (2257)
Runing testcase (2258)
Done testcase (2258)
Runing testcase (2259)
Done testcase (2259)
Runing testcase (2260)
Done testcase (2260)
Runing testcase (2261)
Done testcase (2261)
Runing testcase (2262)
Done testcase (2262)
Runing testcase (2263)
Done testcase (2263)
Runing testcase (2264)
Done testcase (2264)
Runing testcase (2265)
Done testcase (2265)
Runing testcase (2266)
Done testcase (2266)
Runing testcase (2267)
Done testcase (2267)
Runing testcase (2268)
Done testcase (2268)
Runing test

In [66]:
with open("/kaggle/working/datasetRaw/math_train_full_verified.json", "w", encoding='utf-8') as f:
    json.dump({
        "__count__": len(train_data_full_verified),
        "data": train_data_full_verified
    }, f)

In [67]:
while 1:
    time.sleep(1)
    print("True")

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 

## Preprocess math_qa

In [21]:
from datasets import load_dataset

In [22]:
dataset_mathqa = load_dataset("math_qa")

Generating train split:   0%|          | 0/29837 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2985 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4475 [00:00<?, ? examples/s]

Dataset math_qa downloaded and prepared to /root/.cache/huggingface/datasets/math_qa/default/0.1.0/67fc1cc5d22b185002c6fd16e19e4d5215eae01fb04d656bed83204ba6ee55ff. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
train_data_mathqa = []

In [28]:
dataset_mathqa["train"][0]

{'Problem': "the banker ' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?",
 'Rationale': '"explanation : t = 3 years r = 10 % td = ( bg × 100 ) / tr = ( 36 × 100 ) / ( 3 × 10 ) = 12 × 10 = rs . 120 td = ( pw × tr ) / 100 ⇒ 120 = ( pw × 3 × 10 ) / 100 ⇒ 1200 = pw × 3 pw = 1200 / 3 = rs . 400 answer : option a"',
 'options': 'a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d ) rs . 350 , e ) none of these',
 'correct': 'a',
 'annotated_formula': 'divide(multiply(const_100, divide(multiply(36, const_100), multiply(3, 10))), multiply(3, 10))',
 'linear_formula': 'multiply(n2,const_100)|multiply(n0,n1)|divide(#0,#1)|multiply(#2,const_100)|divide(#3,#1)|',
 'category': 'gain'}

In [137]:
mapData = {}
# Analytics
cnt = 0
for i in range(len(dataset_mathqa["train"])):
    singleData = dataset_mathqa["train"][i]
    resRex = re.findall("([\w\d]+)", singleData["annotated_formula"])
    if "const_0_2778" in resRex:
        cnt += 1
        if cnt == 4:
            print(singleData["annotated_formula"])
            print(json.dumps(singleData, indent=4))
            break
#     for j in range(len(resRex)):
#         try:
#             int(resRex[j])
#         except:
# #             resRex2 = re.findall("const_(\d+)", resRex[j])
# #             if len(resRex2) > 0:
# #                 continue
#             if resRex[j] not in mapData:
#                 mapData[resRex[j]] = 1
#             else:
#                 mapData[resRex[j]] += 1

multiply(12, multiply(54, const_0_2778))
{
    "Problem": "a train passes a station platform in 30 sec and a man standing on the platform in 12 sec . if the speed of the train is 54 km / hr . what is the length of the platform ?",
    "Rationale": "\"speed = 54 * 5 / 18 = 15 m / sec . length of the train = 15 * 12 = 180 m . let the length of the platform be x m . then , ( x + 180 ) / 30 = 15 = > x = 270 m . answer : e\"",
    "options": "a ) 227 , b ) 240 , c ) 200 , d ) 178 , e ) 270",
    "correct": "e",
    "annotated_formula": "multiply(12, multiply(54, const_0_2778))",
    "linear_formula": "multiply(n2,const_0_2778)|multiply(n1,#0)|",
    "category": "physics"
}


In [134]:
mapData

{'divide': 43431,
 'multiply': 40911,
 'const_100': 16337,
 'subtract': 23525,
 'add': 29244,
 'const_1': 10834,
 'const_0_2778': 797,
 'const_4': 5041,
 'const_12': 534,
 'const_3': 5429,
 'const_2': 13320,
 'power': 3182,
 'sqrt': 1291,
 'speed': 208,
 'const_10': 3054,
 'const_1000': 1911,
 'floor': 807,
 'const_3_6': 135,
 'const_3600': 589,
 'volume_rectangular_prism': 58,
 'log': 232,
 'choose': 373,
 'square_area': 210,
 'circle_area': 174,
 'circumface': 121,
 'inverse': 1201,
 'const_60': 1208,
 'negate': 397,
 'const_6': 31,
 'rectangle_perimeter': 72,
 'const_1_6': 5,
 'surface_cube': 53,
 'cube_edge_by_volume': 52,
 'volume_cube': 127,
 'surface_cylinder': 8,
 'reminder': 510,
 'const_0_33': 86,
 'original_price_before_loss': 17,
 'const_0_25': 216,
 'lcm': 419,
 'factorial': 955,
 'rectangle_area': 246,
 'rhombus_area': 36,
 'volume_cylinder': 68,
 'const_pi': 241,
 'triangle_area': 45,
 'const_360': 28,
 'square_perimeter': 37,
 'const_52': 13,
 'gcd': 138,
 'p_after_gain

In [138]:
keyMapAnalytics = {}
# Analytics
for i in range(len(dataset_mathqa["train"])):
    singleData = dataset_mathqa["train"][i]
    resRex = re.findall("(.)", singleData["annotated_formula"])
#     if "." in resRex:
#         print(json.dumps(singleData, indent=4))
#         break
    for j in range(len(resRex)):
        if resRex[j] not in keyMapAnalytics:
            keyMapAnalytics[resRex[j]] = 1
        else:
            keyMapAnalytics[resRex[j]] += 1

In [139]:
keyMapAnalytics

{'d': 146449,
 'i': 131598,
 'v': 44948,
 'e': 55565,
 '(': 148442,
 'm': 42633,
 'u': 65635,
 'l': 85410,
 't': 151393,
 'p': 44886,
 'y': 41084,
 'c': 86946,
 'o': 67211,
 'n': 62806,
 's': 87178,
 '_': 63071,
 '1': 65187,
 '0': 95606,
 ',': 143243,
 ' ': 142863,
 '3': 24729,
 '6': 16068,
 ')': 148442,
 'b': 23945,
 'r': 34782,
 'a': 57949,
 '2': 44265,
 '4': 22617,
 '9': 6262,
 '5': 25677,
 '8': 11247,
 '.': 4761,
 '7': 9573,
 'w': 3183,
 'q': 1614,
 'f': 2013,
 'g': 1468,
 'h': 448,
 'x': 32}

In [187]:
train_data_mathqa_raw = []

In [188]:
# Replance const data
i = len(train_data_mathqa_raw)
while i < len(dataset_mathqa["train"]):
    singleData = dataset_mathqa["train"][i]
    
    resRex = re.findall("([\w\d]+)", singleData["annotated_formula"])
    for j in range(len(resRex)):
        try:
            int(resRex[j])
        except:
            # Replance const var
            resRex2 = re.findall("const_(\d+)_(\d+)", resRex[j])
            if len(resRex2) > 0:
                singleData["annotated_formula"] = re.sub("const_{0}_{1}".format(*resRex2[0]), "{0}.{1}".format(*resRex2[0]), singleData["annotated_formula"])
            else:
                resRex2 = re.findall("const_(\d+)", resRex[j])
                if len(resRex2) > 0:
                    singleData["annotated_formula"] = re.sub("const_{0}".format(resRex2[0]), "{0}".format(resRex2[0]), singleData["annotated_formula"])
                    
            resRex2 = re.findall("[0+](\d+)", resRex[j])
    
    singleData["annotated_formula"] = re.sub("([\w]+)\(", r"fun_operator('\1', ", singleData["annotated_formula"])
            
    train_data_mathqa_raw.append(singleData)
    i += 1

In [189]:
train_data_mathqa_raw[433]['annotated_formula']

"fun_operator('multiply', fun_operator('divide', fun_operator('subtract', fun_operator('add', fun_operator('divide', 32, 2), fun_operator('multiply', fun_operator('subtract', 64, 32), 2)), 64), 64), 100)"

In [205]:
dictFun = {
    "divide": "({0} / {1})",
    "multiply": "({0} * {1})",
    "subtract": "({0} - {1})",
    "add": "({0} + {1})",
    "power": "math.pow({0}, {1})",
    "sqrt": "math.sqrt({0})",
    "speed": "({0} / {1})",
    "volume_rectangular_prism": "({0} * {1} * {2})",
    "log": "math.log({0})",
    "choose": "math.comb({0}, {1})",
    "square_area": "({0} * {0})",
    "circle_area": "(math.pi * {0} * {0})",
    "circumface": "(2 * {0})",
    "inverse": "(1 / {0})",
    "negate": "(-{0})",
    "rectangle_perimeter": "(2 * {0} * {1})",
    "surface_cube": "(6 * {0})",
    "cube_edge_by_volume": "pow({0}, 1/3)",
    "volume_cube": "({0} * {0} * {0})",
    "surface_cylinder": "(2 * math.pi * {0} * ({0} + {1}))",
    "reminder": "({0} % {1})",
    "original_price_before_loss": "(({1} * 100) / (100 - {0}))",
    "lcm": "math.lcm({0}, {1})",
    "factorial": "math.factorial({0})",
    "rectangle_area": "({0} * {1})",
    "rhombus_area": "(1 / 2 * {0} * {1})",
    "volume_cylinder": "(math.pi * {0} * {0} * {1})",
    "triangle_area": "(1 / 2 * {0} * {1})",
    "square_perimeter": "(4 * {0})",
    "gcd": "math.gcd({0}, {1})",
    "triangle_area_three_edges": "math.sqrt(({0} + {1} + {2})/2 * (({0} + {1} + {2})/2 - {0}) * (({0} + {1} + {2})/2 - {1}) * (({0} + {1} + {2})/2 - {2}))",
    "quadrilateral_area": "(1 / 2 * {0} * ({1} + {2}))",
    "permutation": "math.perm({0}, {1})",
    "max": "max({0}, {1})",
    "triangle_perimeter": "({0} + {1} + {2})",
    "surface_sphere": "(4 * math.pi * {0} * {0})",
    "volume_sphere": "(4 / 3 * math.pi * {0} * {0} * {0})",
    "diagonal": "math.sqrt({0} * {0} + {1} * {1})",
    "surface_rectangular_prism": "(2 * ({0}*{1} + {1}*{2} + {2}*{0}))",
    "volume_cone": "(1/3 * math.pi * {0} * {0} * {1})",
    "min": "min({0}, {1})"
}

const_pi = 3.141592653589793
const_deg_to_rad = 0.017453292519943295

def fun_operator(name, *args):
    try:
        return dictFun[name].format(*args)
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        print(dictFun[name], "\n", args)
        
    return dictFun[name].format(*args)

In [191]:
train_data_mathqa_raw[40]

{'Problem': 'if x ^ 2 + y ^ 2 = 13 and xy = 3 , then ( x − y ) ^ 2 =',
 'Rationale': '"but you can not take xy + 3 to mean xy = - 3 . . only if xy + 3 = 0 , it will mean xy = - 3 . . rest your solution is perfect and you will get your correct answer as 13 - 2 * 3 = 7 . . answer a"',
 'options': 'a ) 7 , b ) 11 , c ) 14 , d ) 17 , e ) 20',
 'correct': 'a',
 'annotated_formula': "fun_operator('power', 3, 2)",
 'linear_formula': 'power(n3,n0)|',
 'category': 'general'}

In [206]:
train_data_mathqa_raw2 = []
for i in range(len(train_data_mathqa_raw)):
    singleData = train_data_mathqa_raw[i]
    resRex = re.findall("fun\_operator\('([\w]+)'", singleData["annotated_formula"])
    
    ok = True
    for j in range(len(resRex)):
        if resRex[j] not in dictFun:
            ok = False
            break
    if not ok:
        continue
    
    train_data_mathqa_raw2.append({
        **singleData
    })
    
    train_data_mathqa_raw2[-1]['annotated_formula'] = re.sub("([(, _])[0]+(\d+)", r"\1\2", train_data_mathqa_raw2[-1]['annotated_formula'])
 
    try:
        train_data_mathqa_raw2[-1]['annotated_formula'] = eval(train_data_mathqa_raw2[-1]['annotated_formula'])
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        print(i, "\n", singleData['annotated_formula'])
        print(dataset_mathqa["train"][i]['annotated_formula'])
        break

In [207]:
train_data_mathqa_raw2[423]

{'Problem': 'a train leaves delhi at 9 a . m . at a speed of 30 kmph . another train leaves at 2 p . m . at a speed of 40 kmph on the same day and in the same direction . how far from delhi , will the two trains meet ?',
 'Rationale': 'explanation : d = 30 * 5 = 150 rs = 40 – 30 = 10 t = 150 / 10 = 15 d = 40 * 15 = 600 km answer : option c',
 'options': 'a ) 900 km , b ) 800 km , c ) 600 km , d ) 690 km , e ) 609 km',
 'correct': 'c',
 'annotated_formula': '(((30 * (3 + 2)) / (40 - 30)) * 40)',
 'linear_formula': 'add(const_2,const_3)|subtract(n3,n1)|multiply(n1,#0)|divide(#2,#1)|multiply(n3,#3)',
 'category': 'physics'}

### Format stand data_mathqa

In [225]:
def options2choices(options):
    choices = options.split(",")
    for i in range(len(choices)):
        choices[i] = re.sub("(?:([a-zA-Z]) \) )+", r"\1. ", choices[i]).strip()
    return choices

def correct2answer(correct, choices):
    for i in range(len(choices)):
        if correct == choices[i][0]:
            return choices[i]
        
def rationale2explanation(rationale):
    explanation = re.sub("explanation \: ", "", rationale)
    return explanation

In [226]:
train_data_mathqa = []
for i in range(len(train_data_mathqa_raw2)):
    singleData = train_data_mathqa_raw2[i]
    
    train_data_mathqa.append({
        "question": singleData['Problem'],
        "choices": options2choices(singleData['options']),
        "explanation": rationale2explanation(singleData['Rationale']),
        "formula": singleData['annotated_formula'],
        "answer": correct2answer(singleData['correct'], options2choices(singleData['options']))
    })

In [243]:
test_i = 12
print(
    train_data_mathqa[test_i]['question'],
    "\n",
    dataset_mathqa["train"][test_i]['annotated_formula'],
    "\n",
    train_data_mathqa[test_i]['formula'],
    "\n",
    train_data_mathqa[test_i]['explanation'],
    "\n",
    train_data_mathqa[test_i]['answer'],
    "\n",
    eval(train_data_mathqa[test_i]['formula'])
)

the average of 15 result is 60 . average of the first 10 of them is 10 and that of the last 10 is 80 . find the 8 th result ? 
 subtract(add(multiply(10, 10), multiply(10, 80)), multiply(15, 60)) 
 (((10 * 10) + (10 * 80)) - (15 * 60)) 
 "sum of all the 13 results = 15 * 60 = 900 sum of the first 7 of them = 10 * 10 = 100 sum of the last 7 of them = 10 * 80 = 800 so , the 8 th number = 900 + 100 - 800 = 200 . b" 
 b. 200 
 0


In [229]:
with open("/kaggle/working/datasetRaw/train_data_mathqa.json", "w", encoding='utf-8') as f:
    json.dump({
        "__count__": len(train_data_mathqa),
        "data": train_data_mathqa
    }, f)

## Verify sample math qa

In [250]:
modelGemini = genai.GenerativeModel(
    model_name='gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=0.0,
        top_p=0.4,
        top_k=50,
#         max_output_tokens=128,
    )
)

In [251]:
import math
import numpy
import datetime
import itertools

In [252]:
train_data_mathqa_verified = []

In [254]:
# Verify sample
def autoGPTVerifySampleSuper(problem, choices, explanation, expression, expressionResult, answer):
    message = """Verify sample which include [problem description, choise, explantion, true answer, expression, eval expression result] are solvable problem (math problem can solve with logic), correct answer, correct explanation, correct expression (helpful expression, eval expression result provides useful information or aids in calculating the answer). Write a response follow the rule

## Problem
{problem}

## Choices
{choices}

## Explanation
{explanation}

## Expression
{expression}

## Eval expression result
{expressionResult}

## True answer
{answer}

## Rule of response
- No markdown format
- Response must be json
- Response type value must follow the format
- Response must follow format bellow

## Response format
{{
"problem": {{boolean}},
"answer": {{boolean}},
"explanation": {{boolean}},
"expression": {{boolean}}
}}""".format(
        problem = problem,
        choices = "\n".join(choices),
        explanation = explanation,
        expression = expression,
        expressionResult = expressionResult,
        answer = answer
    )
    
    return modelGemini.generate_content(message).text

listKeysNeed = [
    "problem", "answer", "explanation", "expression"
]

i = len(train_data_mathqa_verified)
startTime()
cntSt = 0
timeWait = 1
while i < len(train_data_mathqa):
    singleData = train_data_mathqa[i]
    
    if "ok" in singleData and not singleData["ok"]:
        train_data_full_verified.append(singleData)
        i += 1
        continue
    
    print(f"Runing testcase ({i})")
    
    outVerify = False
    
    try:
        res = autoGPTVerifySampleSuper(singleData['question'], singleData['choices'], singleData['explanation'], singleData['formula'], eval(singleData['formula']), singleData['answer'])
        resRex = re.findall("```json\n([\S\s]+)\n```", res)
        if len(resRex) > 0:
            res = resRex[0]
        
        jsonRes = json.loads(res)
        
        outVerify = True
        for j in range(len(listKeysNeed)):
            if (listKeysNeed[j] not in jsonRes):
                raise Exception("Miss keys in res")
            if not jsonRes[listKeysNeed[j]]:
                outVerify = False
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        print(res)
        
        time.sleep(timeWait)
        timeWait *= 2
        if timeWait > 32:
            break
        continue
    
    timeWait = 1
        
    cntSt += 1
    
    train_data_mathqa_verified.append(singleData)
    train_data_mathqa_verified[-1]["ok"] = outVerify
    
    if not outVerify:
        print("--- False ---")
        print(res)
        print("--- False ---")
        print(json.dumps(singleData, indent=4))
        train_data_mathqa_verified[-1]["okChecker"] = jsonRes
    
    print(f"Done testcase ({i})")
    
    if cntSt%60 == 0:
        deltaTime = getTime()
        if (60 - deltaTime) > -1:
            time.sleep(60 - deltaTime + 2)
        startTime()
        cntSt = 0
    
    i += 1
print("Done all testcase")

Runing testcase (0)
<class 'google.api_core.exceptions.FailedPrecondition'>
400 User location is not supported for the API use.


NameError: name 'res' is not defined

In [218]:
while 1:
    time.sleep(1)

KeyboardInterrupt: 